# Welcome to the Echo package

The "echo" subpackage of murraylab_tools is designed to automate the setup of reactions (mostly TX-TL reactions) with the Echo. The EchoRun class can produce picklists for the Echo, as well as instructions for loading a source plate to go with those instructions, when appropriate. 

There are currently three main modes of operations of the Echo package, distinguished mainly by their inputs:
- **TX-TL Setup Spreadsheet**: Takes a pair of CSV spreadsheets saved from a TX-TL setup spreadsheet (version 2.X). Use this for most TX-TL experiments. 
- **2D Dilution Series in TX-TL**: Takes a Python description of two materials and their dilutions. Use this when you want to run one or more two-way dilution series in TX-TL, for instance when calibrating Mg and K salts. 
- **Association Spreadsheet**: Takes one or more spreadsheets describing the contents of an Echo source plate, plus a simple spreadsheet describing final concentrations of the materials of those source plates in each destination. Use this for non-TX-TL experiments, or if you have a TX-TL experiment with more source materials than can be handled in a TX-TL setup spreadsheet.

*A word of warning*: The quick start examples will help you jump right into setting up an experiment, but they make a number of assumptions about your experiment. Some things you'll want to check before running your own:
- **Reaction Size**: Default is 5 µL.
- **Buffer/Extract Fractions**: Default is 0.24/0.51 (typical for French Press extracts).
- **Source Plate Type**: Default is 384_PP. You probably want this.
- **Source Plate Material**: Default is AQ_BP (buffer-like liquids).
- **Destination Plate Type**: Default is "Nunc_384_black_glassbottom".
- **Destination Plate Size**: The Echo package has no knowledge of the destination plate. *There is no check to keep you from defining picks off the edge of the destination plate*.
- **Controls**: TX-TL experiments come with a negative control. Most TX-TL setup spreadsheets also define a positive control.
- **Dead Volume/Max Volume**: Default dead volume is 20 µL, including loss to meniscus. Default max volume is 65 µL.
- **Volume/Aliquot of Buffer and Extract**: Default is 30 µL extract/aliquot and 37 µL buffer/aliquot.

Additionally, be aware that the TX-TL setup scripts keep track of which source plate wells they've used in a .dat file. If you run those experiments repeatedly, they'll fill up the file and eventually error out when they run out of wells on the plate. 

You can read more about how the echo package works in the next section, "**How it all works**", or you can skip to the example usage sections below that. For more information on tweaking settings, see **Tweaking Settings** at the end of this notebook.

# How it all works

Every project begins with an EchoRun object. This object holds (nearly) all of the information about an experiment, and is ultimately responsible for coordinating plates and materials, and for actually writing picklists and experimental protocols. 

Most EchoRun objects need a SourcePlate object. This object is responsible for keeping track of which wells have been used, and for assigning new wells on the source plate to materials that you'll want to transfer. It will try to assign wells in a way that keeps the same material in a contiguous block, buffered on either side by an empty well for ease of pipetting. SourcePlate objects are also typically associated with a .dat tracking file that lists which wells have been used on the plate. That way, you can use the same source plate over many experiments without having to manually program in which wells are forbidden. The SourcePlate object will read this file to learn what it has available to it, and will automatically write back to the same file whenever the EchoRun object controlling it writes a picklist. 

Materials (DNA, chemicals, water, TX-TL, etc) on a source plate are represented by EchoSourceMaterial objects. An EchoSourceMaterial has a concentration, which is always stored in nM. However, because dsDNA is usually measured in ng/uL, and dsDNA is one of the most common materials used, EchoSourceMaterials by default assume that the concentration set in their constructors is in units of ng/uL. Any EchoSourceMaterial with `length` > 0 will convert that ng/uL concentration into an internal nM concentration. Only if the length of the EchoSourceMaterial is set to 0 will it use its set concentration value directly. This convention appears in several other contexts in the echo package.

An EchoSourceMaterial is always associated with a SourcePlate. The EchoSourceMaterial keeps track of how much of itself has been used, and will request that wells be allocated on the SourcePlate. 

To generate an Echo picklist, you will generally need to do three things:
- Building an EchoRun object. This usually just means setting the name of a source plate tracking file
- Describe the experiment. This almost always means one or more calls to `build_picklist_from_txtl_setup_csvs`, `build_dilution_series`, or `build_picklist_from_association_spreadsheet` from your EchoRun object. What this entails depends on your experiment; TX-TL setup with a setup spreadsheet and association file setups are almost completely defined by external files, while 2D dilution series experiments require some definition in your script. 
- Write the picklist, which is done with a call to `write_picklist` from your EchoRun object. 

For more information, see the examples below.

## TX-TL Setup Spreadsheet

### Quick Start Example

The following creates an Echo picklist and experimental protocol for a variety of fluorescent protein mixes described in "inputs/TX-TL_setup_example.xlsx".

In [2]:
import murraylab_tools.echo as mt_echo
import os.path

# Relevant input and output files. Check these out for examples of input file format.
txtl_inputs  = os.path.join("txtl_setup", "inputs")
txtl_outputs = os.path.join("txtl_setup", "outputs")
stock_file  = os.path.join(txtl_inputs, "TX-TL_setup_example_stocks.csv") # Source materials
recipe_file = os.path.join(txtl_inputs, "TX-TL_setup_example_recipe.csv") # Experimental setup
plate_file  = os.path.join(txtl_inputs, "TX-TL_setup_example_plate.dat")  # Keeps track of wells used 
output_name = os.path.join(txtl_outputs, "TX-TL_setup_example") # Output (both a picklist and a
                                                                # small protocol for building the
                                                                # source plate)

# Build an EchoRun object
txtl_plate = mt_echo.SourcePlate(filename = plate_file)
txtl_echo_calculator = mt_echo.EchoRun(plate = txtl_plate)

# Describe the experiment
txtl_echo_calculator.build_picklist_from_txtl_setup_csvs(stock_file, recipe_file)
# Write results
txtl_echo_calculator.write_picklist(output_name)

 ### More Information

## 2D Dilution Series  in TX-TL

### Quick Start Example

The following creates an Echo picklist and simple experimental protcol for a two-way dilution series of a reporter plasmid and inducer. 

In [3]:
import murraylab_tools.echo as mt_echo
import os.path

# Relevant input and output files. Check these out for examples of input file format.
dilution_inputs  = os.path.join("2D_dilution_series", "inputs")
dilution_outputs = os.path.join("2D_dilution_series", "outputs")
plate_file  = os.path.join(dilution_inputs, "dilution_setup_example_plate.dat") # Keeps track of wells used
output_name = os.path.join(dilution_outputs, "dilution_setup_example") # Output (both a picklist and a
                                                               # small protocol for building the
                                                               # source plate)

# Build an EchoRun object
dilution_plate = mt_echo.SourcePlate(filename = plate_file)
dilution_echo_calculator = mt_echo.EchoRun(plate = dilution_plate)

# Set final concentrations of two materials
gfp_final_concentrations = range(0,6,1) # in nM
atc_final_concentrations = range(0,100,10) # in ng/uL

# Define reporter plasmid material
gfp_conc = 294  # Concentration in ng/uL
gfp_len  = 3202 # Size of DNA in bp
gfp = mt_echo.EchoSourceMaterial('GFP Plasmid', gfp_conc, gfp_len, dilution_plate)

# Define inducer material
atc_conc = 1000 # Concentration in ng/uL (important that this matches the units of the final concentrations)
atc_len  = 0    # This isn't dsDNA, so it has 0 length. 
atc = mt_echo.EchoSourceMaterial("ATc", atc_conc, atc_len, dilution_plate)

# Plan out the experiment
starting_well = "D2"
dilution_echo_calculator.build_dilution_series(gfp, atc, gfp_final_concentrations,
                                               atc_final_concentrations, starting_well)
# Write results
dilution_echo_calculator.write_picklist(output_name)

### More Information

## Association Spreadsheet

### Quick Start Example

The following creates an Echo picklist for an automated PCR setup with three sets of primers to be applied individually to three different plasmids, as defined in a pair of CSV files (one defining the source plate, one describing what should go in the destination plates). 

In [4]:
import murraylab_tools.echo as mt_echo
import os.path

# Relevant input and output files. Check these out for examples of input file format.
assoc_inputs  = os.path.join("association_list", "inputs")
assoc_outputs = os.path.join("association_list", "outputs") 
stock_file = os.path.join(assoc_inputs, 'association_source_sheet.csv')
assoc_file = os.path.join(assoc_inputs, 'association_final_sheet.csv')
assoc_name = os.path.join(assoc_outputs, 'association_example')

# Build an EchoRun object
assoc_echo_calculator = mt_echo.EchoRun()
assoc_echo_calculator.rxn_vol = 50000 # PCR is large-volume!

# Define which column of the source file is what
name_col  = 'B'
conc_col  = 'C'
len_col   = 'D'
well_col  = 'A'
plate_col = 'E'

# Define the source plate based on the stock file.
assoc_echo_calculator.load_source_plate(stock_file, name_col, conc_col,
                                        len_col, well_col, plate_col)

# Build a protocol, based on the association file.
assoc_echo_calculator.build_picklist_from_association_spreadsheet(assoc_file,
                                                                  well_col)

# Write the picklist
assoc_echo_calculator.write_picklist(assoc_name)

### More Information

# Tweaking Settings

In [5]:
import murraylab_tools.echo as mt_echo
import os.path

plate_file = os.path.join("tweaking", "plate_file_example.dat")

# Build an EchoRun object
example_plate = mt_echo.SourcePlate(filename = plate_file)
example_echo_calculator = mt_echo.EchoRun(plate = example_plate)

#### To change the reaction volume:

Reaction volume is a property of an EchoRun object.

In [6]:
example_echo_calculator.rxn_vol = 10.5

Make sure to run this before running `build_picklist_from_association_spreadsheet` or `build_dilution_series`. You almost certainly shouldn't do this at all when using `build_picklist_from_txtl_setup_csvs`, because that function will automatically extract a reaction volume from the setup spreadsheet.

#### To change the buffer/extract fractions:

This is really only relevant for the 2D dilution series TX-TL setup (`build_dilution_series`) -- TX-TL setup from a spreadsheet pulls the extract fraction from the spreadsheet, and the association spreadsheet method has no knowledge of TX-TL. Accordingly, the extract fraction is an optional argument in `build_dilution_series`.

In [7]:
extract_fraction = 0.17
example_echo_calculator.build_dilution_series(gfp, atc, gfp_final_concentrations,
                                              atc_final_concentrations, starting_well,
                                              extract_fraction)

#### To change the source plate type/material type:

Source plate types and material types are set as optional arguments in the constructor of a SourcePlate object. The type and material of a source plate are both set by a string, which can be any string that the Echo Plate Reformat software will recognize.

In [8]:
plate_type = "384PP_AQ_BP" # This is actually the default plate value
retyped_example_plate = mt_echo.SourcePlate(filename = plate_file, SPtype = plate_type)
another_example_echo_calculator = mt_echo.EchoRun(plate = retyped_example_plate)

#### To change the source plate name:

Source plate names are set much like source plate types with the argument `SPname`. In addition, as a shorthand, you can set `SPname` to be a number N, in which case the plate will be named `Source[N]`. 

In [9]:
plate_name = "FirstPlate"
renamed_example_plate = mt_echo.SourcePlate(filename = plate_file, SPname = plate_name)
yet_another_example_echo_calculator = mt_echo.EchoRun(plate = renamed_example_plate)

**To change the destination plate type:**

Destination plate types are determined and stored directly in the EchoRun object. The destination plate type can be set by the optional argument `DPtype` in the constructor of an EchoRun object, or set manually any time before calling `write_picklist` on that EchoRun.

In [10]:
calculator_with_odd_destination = mt_echo.EchoRun(plate = example_plate, DPtype = "some_kind_of_plate")
calculator_with_odd_destination.DPtype = "Nunc_384_black_glassbottom" # Just kidding!
# ...
# calculator_with_odd_destination.write_picklist(...)
#...

**To change dead volume and max volume**:

You probably shouldn't do this. If you absolutely must squeeze every last bit of efficiency out of your source wells, you can set the `dead_volume` and `max_volume` variables, which are static variables in the murraylab_tools.echo package. If you change them, also make sure to set the static variable `usable_volume`, which defines the volume of material in a well that can actually be used by the Echo (this is normally calculated from `dead_volume` and `max_volume` at package import). Also, you should do this before running any experimental protocol function.

In [11]:
from murraylab_tools.echo.echo_functions import dead_volume, max_volume, usable_volume
dead_volume   = 10000 # Volume in nL!
max_volume    = 75000 # Volume in nL!
usable_volume = max_volume - dead_volume # Don't forget to re-calculate this!

**To change buffer/extract aliquot size:**

Buffer and extract aliquot size, like destination plate type, are managed by an EchoRun object, and can be set either manually (before making a `write_picklist` call) or in the constructor of that EchoRun object. 

Note that both aliquot sizes are in units of nL, not uL.

In [12]:
custom_aliquot_calculator = mt_echo.EchoRun(plate = example_plate, 
                                            extract_per_aliquot = 50000,
                                            buffer_per_aliquot = 100000)
custom_aliquot_calculator.extract_per_aliquot = 75000
custom_aliquot_calculator.buffer_per_aliquot  = 150000
# ...
# calculator_with_odd_destination.write_picklist(...)
#...